In [1]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
mturk_path = Path("../mturk/results")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

%load_ext autoreload
%autoreload 2

In [2]:
ls(mturk_path)

['../mturk/results/Batch_4258572_batch_results.csv',
 '../mturk/results/Batch_4280306_batch_results.csv']

In [3]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    return df.set_index("AssignmentId")
batch = 4280306
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,events,feedback,id,status
AssignmentId,,,,,,,
354GIDR5ZH1VMLMJ416WJOK9FKS00Z,32L724R86QF7UDAPZA5RF9PQESGIPI,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A2VRDE2FHCBMF8,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},52446,1
3T3IWE1XGCIKQNO9J1OLI5DWQ1PTQ9,3T8DUCXY1S1CB4ATCX8Y7QRKK5ET9F,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A3P4JJWKLKQUD7,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},19547,1
3F6HPJW4JJVVQLQICV3V97MSEVN2W0,389A2A305TDDTTK5QFUFP5QNTFL0CS,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A72LC42LU78IP,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},95554,1
3NXNZ5RS1GSR0QAGPWAYREVPR51970,389A2A305TDDTTK5QFUFP5QNTFL0CS,3PUZG2R74XFTJN1AOYZP8I3IL1754O,AVINXZZV3FNG7,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},95554,1
3M0NZ3JDP7TRJDEA51P06IJMEZOZ5D,3C8QQOM6KUWH3SVR7ICKYSG69IFLIN,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A2W0AMLFOH96E9,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},97806,1
...,...,...,...,...,...,...,...
3X3OR7WPZ5VQ16G8OY5UANIF1GEL8F,31S7M7DAHLL7C6B21UBOCZIFKZVLT4,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A140QYQ41DKHPF,"[[],[],[]]",{},35564,1
3ND9UOO81QX1R7CQM1L0O3L1KPPLW7,3J9UN9O9K8N017QB07B1C93IJZTJ0O,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A2G7DTL156PTKA,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},136399,1
3B1NLC6UG5RVL61OZOAPJH8XVBKPGU,3OYHVNTV6YTGF8OCKDDO3PZDWR9OKI,3PUZG2R74XFTJN1AOYZP8I3IL1754O,A12FTSX85NQ8N9,"[[],[],[]]",{},218285,1


In [4]:
assignment="  31HQ4X3T3Y58KQVH2WBZREIGECRLSQ".strip()
idx = df.loc[assignment, "id"]
status = df.loc[assignment, "status"]
display_html(f'<a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">{idx} - {status}</a>')

In [33]:
str(next(mturk_path.rglob("*4258572*.csv")))

'../mturk/results/Batch_4258572_batch_results.csv'